In [96]:
import re
import io
import requests
import pandas as pd
from fuzzywuzzy import fuzz, process
from states import us_state_to_abbrev, abbrev_to_us_state

In [100]:
raypac_house_2020 = pd.read_csv("raypac-house-2020.csv")
raypac_senate_2020 = pd.read_csv("raypac-senate-2020.csv")
jan6_house = pd.read_csv("jan6-house-deniers.csv")
jan6_senate = pd.read_csv("jan6-senate-deniers.csv")
url = "https://raw.githubusercontent.com/fivethirtyeight/data/master/election-deniers/fivethirtyeight_election_deniers.csv"
res = requests.get(url)
ballot_deniers = pd.read_csv(io.StringIO(res.content.decode('utf-8')))
ballot_deniers = ballot_deniers.assign(party="r")

In [102]:
raypac_2020 = pd.concat([raypac_house_2020.assign(office="Representative"), raypac_senate_2020.assign(office="Senator")], ignore_index=True)
jan6_congress = pd.concat([jan6_house.assign(office="Representative"), jan6_senate.assign(office="Senator")], ignore_index=True)

In [103]:
def clean_df(df):
    # Drop to lowercase
    clean = df.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)
    clean.columns = df.columns.str.lower()

    # Convert currency string to float
    if "total" in clean.columns:
        clean["total_float"] = clean["total"].apply(lambda x: float(re.sub(r'[^\d.]', '', x)))

    # Fill name column
    if not "name" in clean.columns:
        clean["name"] = "candidate"
    
    # Trim party names to first letter
    clean["party"] = clean["party"].apply(lambda x: x[0])

    # State full name or abbreviation to 2 letter code
    state_codes = []
    for s in clean["state"]:
        if len(s) == 2:
            # State is already in two-letter code form
            state_codes.append(s)
        else:
            try:
                # Try for exact match (will not occur in current setup due to capitalization)
                state_codes.append(us_state_to_abbrev[s].lower())
            except KeyError:
                # Look for closest state match
                closest_state_name = process.extractOne(s, list(us_state_to_abbrev.keys()))[0]
                state_codes.append(us_state_to_abbrev[closest_state_name].lower())
    clean["state_code"] = state_codes
    return clean

In [104]:
raypac_2020_clean = clean_df(raypac_2020)
jan6_congress_clean = clean_df(jan6_congress)
ballot_deniers_clean = clean_df(ballot_deniers)

In [105]:
jan6_deniers = jan6_congress_clean.query("vote=='yea'")

In [106]:
def fuzzy_filter(df_1, df_2, key1, key2, threshold=90, guarantee=["party", "state_code"]):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    nearest_match = df_1[key1].apply(lambda x: process.extract(x, s))    
    match = nearest_match.apply(lambda x: x[0][0] if x[0][1] >= threshold else None)
    filtered = df_1.copy()
    filtered["matched_denier"] = match
    filtered = filtered[filtered["matched_denier"].notna()]
    for key in guarantee:
        for ix, row in filtered.iterrows():
            guarantee_value = df_2.loc[df_2["name"] == row["matched_denier"]][key].iloc[0]
            if not row[key] == guarantee_value:
                filtered = filtered.drop(ix)

    return filtered


In [107]:
raypac_2020_deniers = fuzzy_filter(raypac_2020_clean, jan6_deniers, 'candidate', 'name')

In [108]:
raypac_2020_deniers

,candidate,party,state,total,cycle,office,total_float,name,state_code,matched_denier
1,devin nunes,r,calif,"$20,000",2020,representative,20000.0,candidate,ca,nunes
2,robert b aderholt,r,ala,"$10,000",2020,representative,10000.0,candidate,al,aderholt
4,richard w allen,r,ga,"$10,000",2020,representative,10000.0,candidate,ga,allen
5,brian babin,r,texas,"$10,000",2020,representative,10000.0,candidate,tx,babin
7,jim banks,r,ind,"$10,000",2020,representative,10000.0,candidate,in,banks
19,tom cole,r,okla,"$10,000",2020,representative,10000.0,candidate,ok,cole
34,michael guest,r,miss,"$10,000",2020,representative,10000.0,candidate,ms,guest
35,vicky hartzler,r,mo,"$10,000",2020,representative,10000.0,candidate,mo,hartzler
48,kevin mccarthy,r,calif,"$10,000",2020,representative,10000.0,candidate,ca,mccarthy
54,steven palazzo,r,miss,"$10,000",2020,representative,10000.0,candidate,ms,palazzo


In [109]:
total_donations = raypac_2020_deniers["total_float"].sum()
print(f"Raytheon Technologies via RAYPAC donated ${total_donations:.2f} to members of congress that voted against certifying the 2020 election in the 2019-2020 election cycle")

Raytheon Technologies via RAYPAC donated $235500.00 to members of congress that voted against certifying the 2020 election in the 2019-2020 election cycle
